In [1]:
##Estimación de indicadores de vivienda
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr, haven, googledrive)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin1<-"_ns_"
fin2<-"_csv.zip"

In [4]:
years<-c("2016", "2018", "2020", "2022")

In [5]:
# Función genérica para descargar, descomprimir, renombrar y eliminar
procesar_archivo <- function(year, tipo, url_base, segunda_parte, fin1, fin2, carpeta_destino) {
  # Construir URL y ruta de destino
  url <- paste0(url_base, year, "/", segunda_parte, year, fin1, tipo, fin2)
  destfile <- paste0(carpeta_destino, "/", year, fin1, tipo, fin2)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = carpeta_destino)

  # Renombrar el archivo descomprimido
  file.rename(
    from = paste0(carpeta_destino, "/", tipo, ".csv"),
    to = paste0(carpeta_destino, "/", tipo, "_", year, ".csv")
  )

  # Eliminar el archivo descargado
  file.remove(destfile)
}

In [6]:
# Iterar sobre los años y procesar cada tipo de archivo
for (i in 1:length(years)) {
  procesar_archivo(years[i], "poblacion", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "viviendas", url_basica, segunda_parte, fin1, fin2, "data")
  procesar_archivo(years[i], "trabajos", url_basica, segunda_parte, fin1, fin2, "data")
}

### Función de estimación

In [19]:
estimacion <- function(path, variable) {
  # Cargar los datos de población
  datos <- read_csv(path, show_col_types = FALSE)

  # Si son datos de 2016 a 2020, unir con la tabla de viviendas
  if (substr(path, 16, 19) %in% c("2016", "2018", "2020")) {
    # Cargar tabla viviendas
    vivienda <- read_csv(str_replace(path, "poblacion", "viviendas"), show_col_types = FALSE)

    # Pegar factor, upm y est_dis de la tabla vivienda
    datos <- datos %>%
      left_join(vivienda %>% select(folioviv, factor, upm, est_dis), by = "folioviv")
  }

  # Unir con la tabla trabajos si es "ocupacion" o si la persona debe ser identificada como indígena
  if (variable %in% c("ocupacion", "ocupacion_mujer")) {
    # Cargar tabla trabajos
    trabajos <- read_csv(str_replace(path, "poblacion", "trabajos"), show_col_types = FALSE)

    # Pegar datos relevantes de la tabla trabajos
    datos <- datos %>%
      left_join(trabajos %>% select(folioviv, foliohog, numren, id_trabajo, trapais),
                by = c("folioviv", "foliohog", "numren"))
  }

  # Crear variable cve_ent
  datos <- datos %>%
    mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es CDMX (09)
    filter(cve_ent == "09")

  # Definir diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estimar población hablante indígena en CDMX
  if (variable == "hablante") {
    hablante <- mydesign %>%
      group_by(hablaind) %>%
      summarise(
        poblacion = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = round(porcentaje * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(hablaind == 1)

    return(hablante)
  }

    # Estimar población hablante indígena en CDMX
  if (variable == "hablante_indigena") {
    hablante_ind <- mydesign %>%
      group_by(etnia,hablaind) %>%
      summarise(
        poblacion = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = round(porcentaje * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(etnia == 1)%>%
filter(hablaind == 1)

    return(hablante_ind)
  }


  # Estimar población ocupada, considerando si la persona es indígena
  if (variable == "ocupacion") {
    ocupacion <- mydesign %>%
      group_by(trapais,etnia) %>%
      summarise(
        ocupados = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = round(porcentaje * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
      filter(trapais == 1)%>%
filter(etnia == 1)
    return(ocupacion)
  }



  # Estimar población ocupada, considerando si la persona es indígena
  if (variable == "ocupacion_mujer") {
    ocupacion_muj <- mydesign %>%
      filter(trapais==1)%>%
      group_by(etnia,sexo) %>%
      summarise(
        ocupados = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = round(porcentaje * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
        #Filtro población indígena
filter(etnia == 1) %>%
  #Filtro mujer
filter(sexo==2)
    return(ocupacion_muj)
  }





  # Estimar población analfabeta
  if (variable == "analfabetismo") {
    analfa <- mydesign %>%
      group_by(etnia,alfabetism) %>%
      summarise(
        analfabetas = survey_total(vartype = "cv"),
        porcentaje = survey_prop(vartype = "cv")
      ) %>%
      mutate(porcentaje = round(porcentaje * 100, 2)) %>%
      # Añadir año
      mutate(year = substr(path, 16, 19)) %>%
filter(etnia == 1)%>%
filter(alfabetism == 2)
    return(analfa)
  }

}


#### Población hablante indígena en CDMX

In [8]:
tabla_hablante<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "hablante"
)
print("Población hablante de lengua indígena en la CDMX")
tabla_hablante

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


[1] "Población hablante de lengua indígena en la CDMX"


hablaind,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,129097,0.1508605,1.41,0.1505873,2016
1,127656,0.2158292,1.38,0.2139413,2018
1,154656,0.1536397,1.67,0.1530732,2020
1,134098,0.1518493,1.43,0.1521503,2022


#### Población hablante y autodescripción indígena en CDMX

In [9]:
tabla_hablante_indigena<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "hablante_indigena"
)
print("Población según autodescripción y condición de hablante de lengua indígena en la CDMX")
tabla_hablante_indigena

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `porcentaje = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `hablaind = NA`.
Caused by warning:
! glm.fit: algorithm did not converge”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `porcentaje = survey_prop(vartype = "cv")`.
ℹ In group 5: `etnia = NA` and `hablaind = NA`.
Caused by warning:
! glm.fit: algorithm did no

[1] "Población según autodescripción y condición de hablante de lengua indígena en la CDMX"


etnia,hablaind,poblacion,poblacion_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,110147,0.1701360,8.81,0.1710783,2016
1,1,107151,0.2531187,7.83,0.2424770,2018
1,1,139719,0.1634319,10.03,0.1551507,2020
1,1,116757,0.1632034,10.58,0.1590305,2022


#### Población ocupada indígena en CDMX

In [10]:
tabla_ocupacion<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "ocupacion"
)
print("Población que se autodescribe como indígena y que se encuentra ocupada en la CDMX")
tabla_ocupacion

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

[1] "Población que se autodescribe como indígena y que se encuentra ocupada en la CDMX"


trapais,etnia,ocupados,ocupados_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,701096,0.08687768,15.24,0.08163274,2016
1,1,716466,0.08822712,15.83,0.08266348,2018
1,1,684724,0.06773705,16.27,0.06419507,2020
1,1,550736,0.08300888,11.86,0.08205935,2022


####   Población ocupada indígena mujer en CDMX

In [21]:
tabla_ocupacion_mujer<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "ocupacion_mujer"
)
print("Población que se autodescribe como indígena que es mujer y que se encuentra ocupada en la CDMX")
tabla_ocupacion_mujer

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

[1] "Población que se autodescribe como indígena que es mujer y que se encuentra ocupada en la CDMX"


etnia,sexo,ocupados,ocupados_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2,308308,0.10074117,43.98,0.04794138,2016
1,2,321598,0.10590241,44.89,0.04630140,2018
1,2,311122,0.07667507,45.44,0.04340553,2020
1,2,229755,0.09928867,41.72,0.05397506,2022


####   Población analfabeta indígena en CDMX

In [ ]:
tabla_analfabetismo<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv"),
  estimacion,
  variable = "analfabetismo"
)
print("Población que se autodescribe como indígena y que es analfabeta en la CDMX")

tabla_analfabetismo

### Calcular porcentaje de población indígena que se encuentra en situación de pobreza


Para esto, se utilizan las bases generadas por Evalúa CDMX. Es importante señalar que, por practicidad, se trabaja con el producto terminado, no obstante, se buscará en próximas fechas replicar dicha metodología utilizando R ya que actualmente, el programa de cálculo se encuentra en STATA.

In [ ]:
  #Conectar a google drive
drive_auth(email = "claudiodanielpc@gmail.com",cache = FALSE)

In [ ]:
carpeta<-as_id(drive_get("https://drive.google.com/drive/folders/1TBfoik4w4DWhwFqWeZFgkZvrBdDbqzku"))


archivos<-drive_ls(carpeta)
print(archivos)

In [ ]:
archivos[1]

In [ ]:
estimacion_pobreza <- function(file_id, file_name) {
  # Descarga de datos
  local_path <- file.path("data", file_name)
  drive_download(as_id(file_id), path = local_path, overwrite = TRUE)

  # Leer datos
  datos <- read_dta(local_path)

  # Añadir año
  year <- str_extract(file_name, "\\d{4}")


  # Diseño muestral
  mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estima pobreza
  pobreza <- mydesign %>%
    #Filtrar datos para CDMX
  filter(entidad==9)%>%
    #Filtar grupo de población en pobreza
    filter(pobreza_mmip == 1) %>%
    group_by(etnia)%>%
    summarise(
      poblacion = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = as.numeric(year))%>%
             #Filtro de población indígena
             filter(etnia==1)

  return(pobreza)
}


In [ ]:
tabla_pobreza <- map_dfr(
  seq_len(nrow(archivos)),
  ~ estimacion_pobreza(archivos$id[.x], archivos$name[.x])
)%>%
  arrange(year)

print("Estimación de población indígena en situación de pobreza en la CDMX")
tabla_pobreza